In [1]:
# import all the packages needed
import re
import os
import glob
import zipfile
import requests
from urllib.request import urlretrieve
import json
import pandas as pd
import numpy as np

In [2]:
%cd /Users/yaouh/525_milestone/milestone_1
## Please change it to the location of your choice

c:\Users\yaouh\525_milestone\milestone_1


In [3]:
# Necessary metadata
article_id = 14096681  
# Beuzen, Tomas (2021): Daily rainfall over NSW, Australia. figshare. Dataset. https://doi.org/10.6084/m9.figshare.14096681.v3
url = f"https://api.figshare.com/v2/articles/{article_id}"
headers = {"Content-Type": "application/json"}
output_directory = "rainforest/" # change the output directory

In [4]:
response = requests.request("GET", url, headers=headers)
data = json.loads(response.text)  # this contains all the articles data
files = data["files"]             # this is just the data about the files
files

[{'id': 26579150,
  'name': 'daily_rainfall_2014.png',
  'size': 58863,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26579150',
  'supplied_md5': 'fd32a2ffde300a31f8d63b1825d47e5e',
  'computed_md5': 'fd32a2ffde300a31f8d63b1825d47e5e'},
 {'id': 26579171,
  'name': 'environment.yml',
  'size': 192,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26579171',
  'supplied_md5': '060b2020017eed93a1ee7dd8c65b2f34',
  'computed_md5': '060b2020017eed93a1ee7dd8c65b2f34'},
 {'id': 26586554,
  'name': 'README.md',
  'size': 5422,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26586554',
  'supplied_md5': '61858c6cc0e6a6d6663a7e4c75bbd88c',
  'computed_md5': '61858c6cc0e6a6d6663a7e4c75bbd88c'},
 {'id': 26766812,
  'name': 'data.zip',
  'size': 814041183,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26766812',
  'supplied_md5': 'b517383f76e77bd03755a63a8f

In [5]:
%%time
files_to_dl = ["data.zip"]  # data.zip
for file in files:
    if file["name"] in files_to_dl:
        os.makedirs(output_directory, exist_ok=True)
        urlretrieve(file["download_url"], output_directory + file["name"])

CPU times: total: 4.48 s
Wall time: 1min 16s


In [6]:
%%time
with zipfile.ZipFile(os.path.join(output_directory, "data.zip"), 'r') as f:
    f.extractall(output_directory)

CPU times: total: 13.8 s
Wall time: 14.1 s


In [7]:
#%ls -ltr rainforest

In [8]:
files = glob.glob('rainforest/*.csv')
files

['rainforest\\ACCESS-CM2_daily_rainfall_NSW.csv',
 'rainforest\\ACCESS-ESM1-5_daily_rainfall_NSW.csv',
 'rainforest\\AWI-ESM-1-1-LR_daily_rainfall_NSW.csv',
 'rainforest\\BCC-CSM2-MR_daily_rainfall_NSW.csv',
 'rainforest\\BCC-ESM1_daily_rainfall_NSW.csv',
 'rainforest\\CanESM5_daily_rainfall_NSW.csv',
 'rainforest\\CMCC-CM2-HR4_daily_rainfall_NSW.csv',
 'rainforest\\CMCC-CM2-SR5_daily_rainfall_NSW.csv',
 'rainforest\\CMCC-ESM2_daily_rainfall_NSW.csv',
 'rainforest\\EC-Earth3-Veg-LR_daily_rainfall_NSW.csv',
 'rainforest\\FGOALS-f3-L_daily_rainfall_NSW.csv',
 'rainforest\\FGOALS-g3_daily_rainfall_NSW.csv',
 'rainforest\\GFDL-CM4_daily_rainfall_NSW.csv',
 'rainforest\\GFDL-ESM4_daily_rainfall_NSW.csv',
 'rainforest\\INM-CM4-8_daily_rainfall_NSW.csv',
 'rainforest\\INM-CM5-0_daily_rainfall_NSW.csv',
 'rainforest\\KIOST-ESM_daily_rainfall_NSW.csv',
 'rainforest\\MIROC6_daily_rainfall_NSW.csv',
 'rainforest\\MPI-ESM-1-2-HAM_daily_rainfall_NSW.csv',
 'rainforest\\MPI-ESM1-2-HR_daily_rainfall_

In [9]:
sample1 = pd.read_csv('rainforest/SAM0-UNICON_daily_rainfall_NSW.csv', index_col=0)
cols = sample1.columns.to_list()
cols = ['time'] + cols
cols

['time', 'lat_min', 'lat_max', 'lon_min', 'lon_max', 'rain (mm/day)']

In [10]:
%%time
## Combine data CSVs into a single CSV using pandas
files = glob.glob('rainforest/*.csv')
df = pd.concat((pd.read_csv(file, index_col=False, 
                            usecols=cols,
                            #low_memory = False
                           )
                .assign(model=re.findall('\\\\(.*?)_', file)[0]) # add the model column
                #find the model name before the first underscore
                for file in files if file != 'rainforest\\observed_daily_rainfall_SYD.csv'
                # do not include observed_daily_rainfall_SYD.csv
               )
              )
df.to_csv("rainforest/combined.csv")

CPU times: total: 6min 47s
Wall time: 6min 53s


In [11]:
df.head()

,time,lat_min,lat_max,lon_min,lon_max,rain (mm/day),model
0,1889-01-01 12:00:00,-36.25,-35.0,140.625,142.5,3.293256e-13,ACCESS-CM2
1,1889-01-02 12:00:00,-36.25,-35.0,140.625,142.5,0.000000e+00,ACCESS-CM2
2,1889-01-03 12:00:00,-36.25,-35.0,140.625,142.5,0.000000e+00,ACCESS-CM2
3,1889-01-04 12:00:00,-36.25,-35.0,140.625,142.5,0.000000e+00,ACCESS-CM2
4,1889-01-05 12:00:00,-36.25,-35.0,140.625,142.5,1.047658e-02,ACCESS-CM2


In [12]:
df32 = df.astype('float32', errors='ignore')

In [13]:
# changing the dytpe of the data
print(f"Memory usage with float64: {df.memory_usage().sum() / 1e6:.2f} MB")
print(f"Memory usage with float32: {df32.memory_usage().sum() / 1e6:.2f} MB")

Memory usage with float64: 3997.94 MB
Memory usage with float32: 2748.59 MB


In [14]:
# changing the dytpe of the data, value_counts()
# print(f"Memory usage with float64: {df.value_counts().memory_usage() / 1e6:.2f} MB")
# print(f"Memory usage with float32: {df32.value_counts().memory_usage() / 1e6:.2f} MB")

In [15]:
# %%time
# df.value_counts()

In [16]:
# %%time
# df32.value_counts()

In [17]:
%%time
df['model'].unique()

CPU times: total: 3.11 s
Wall time: 3.17 s


array(['ACCESS-CM2', 'ACCESS-ESM1-5', 'AWI-ESM-1-1-LR', 'BCC-CSM2-MR',
       'BCC-ESM1', 'CanESM5', 'CMCC-CM2-HR4', 'CMCC-CM2-SR5', 'CMCC-ESM2',
       'EC-Earth3-Veg-LR', 'FGOALS-f3-L', 'FGOALS-g3', 'GFDL-CM4',
       'GFDL-ESM4', 'INM-CM4-8', 'INM-CM5-0', 'KIOST-ESM', 'MIROC6',
       'MPI-ESM-1-2-HAM', 'MPI-ESM1-2-HR', 'MPI-ESM1-2-LR', 'MRI-ESM2-0',
       'NESM3', 'NorESM2-LM', 'NorESM2-MM', 'SAM0-UNICON', 'TaiESM1'],
      dtype=object)

In [18]:
%%time
df32['model'].unique()

CPU times: total: 2.94 s
Wall time: 2.93 s


array(['ACCESS-CM2', 'ACCESS-ESM1-5', 'AWI-ESM-1-1-LR', 'BCC-CSM2-MR',
       'BCC-ESM1', 'CanESM5', 'CMCC-CM2-HR4', 'CMCC-CM2-SR5', 'CMCC-ESM2',
       'EC-Earth3-Veg-LR', 'FGOALS-f3-L', 'FGOALS-g3', 'GFDL-CM4',
       'GFDL-ESM4', 'INM-CM4-8', 'INM-CM5-0', 'KIOST-ESM', 'MIROC6',
       'MPI-ESM-1-2-HAM', 'MPI-ESM1-2-HR', 'MPI-ESM1-2-LR', 'MRI-ESM2-0',
       'NESM3', 'NorESM2-LM', 'NorESM2-MM', 'SAM0-UNICON', 'TaiESM1'],
      dtype=object)

In [19]:
%%time
df32.columns.to_list()

CPU times: total: 0 ns
Wall time: 0 ns


['time', 'lat_min', 'lat_max', 'lon_min', 'lon_max', 'rain (mm/day)', 'model']

In [20]:
%%time
df.describe()

CPU times: total: 12.8 s
Wall time: 12.9 s


,lat_min,lat_max,lon_min,lon_max,rain (mm/day)
count,5.924854e+07,6.246784e+07,5.924854e+07,6.246784e+07,5.924854e+07
mean,-3.310482e+01,-3.197757e+01,1.469059e+02,1.482150e+02,1.901170e+00
std,1.963549e+00,1.992067e+00,3.793784e+00,3.809994e+00,5.585735e+00
min,-3.646739e+01,-3.600000e+01,1.406250e+02,1.412500e+02,-3.807373e-12
25%,-3.486911e+01,-3.366221e+01,1.434375e+02,1.450000e+02,3.838413e-06
50%,-3.300000e+01,-3.204188e+01,1.468750e+02,1.481250e+02,6.154947e-02
75%,-3.140170e+01,-3.015707e+01,1.501875e+02,1.513125e+02,1.020918e+00
max,-2.990000e+01,-2.790606e+01,1.537500e+02,1.556250e+02,4.329395e+02


In [21]:
%%time
df32.describe()

CPU times: total: 7.83 s
Wall time: 7.92 s


,lat_min,lat_max,lon_min,lon_max,rain (mm/day)
count,5.924854e+07,6.246784e+07,5.924854e+07,6.246784e+07,5.924854e+07
mean,-3.310463e+01,-3.197747e+01,1.469054e+02,1.482152e+02,1.901175e+00
std,1.963549e+00,1.992067e+00,3.793784e+00,3.809994e+00,5.585735e+00
min,-3.646739e+01,-3.600000e+01,1.406250e+02,1.412500e+02,-3.807373e-12
25%,-3.486911e+01,-3.366221e+01,1.434375e+02,1.450000e+02,3.838413e-06
50%,-3.300000e+01,-3.204189e+01,1.468750e+02,1.481250e+02,6.154947e-02
75%,-3.140170e+01,-3.015707e+01,1.501875e+02,1.513125e+02,1.020918e+00
max,-2.990000e+01,-2.790606e+01,1.537500e+02,1.556250e+02,4.329395e+02


In [22]:
use_cols = ['lat_min', 'lat_max', 'lon_min', 'lon_max', 'rain (mm/day)']

In [23]:
import os
os.environ['R_HOME'] = '/Users/yaouh/miniconda3/envs/525_2023/Lib/R'